In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import plotting

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('/kaggle/input/adidas-vs-nike/Adidas Vs Nike.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df=df.rename(columns={'Product ID': 'Product_ID', 'Listing Price': 'Listing_Price','Sale Price':'Sale_Price','Last Visited':'Last_Visited'})

In [ ]:
df['Brand'].unique()

In [ ]:
df=df.replace('Adidas Adidas ORIGINALS', 'Adidas ORIGINALS')

In [ ]:
df.describe()

# 1. Brand Analysis

# 1) Numbers of each Brand

In [ ]:
df.groupby('Brand')['Product Name'].count().plot.barh()

# 2) Listing Price by Brand

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(df[df["Brand"]=="Adidas ORIGINALS"].Listing_Price,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Adidas CORE / NEO"].Listing_Price,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Adidas SPORT PERFORMANCE"].Listing_Price,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Nike"].Listing_Price,kde=False,rug=False)
plt.legend(labels=['Adidas ORIGINALS', 'Adidas CORE / NEO','Adidas SPORT PERFORMANCE', 'Nike'])
plt.show()

# What are the listing price 0 products in NIKE ?

In [ ]:
df[df['Listing_Price']==0]

# 3) Sale Price by Brand

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(df[df["Brand"]=="Adidas ORIGINALS"].Sale_Price,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Adidas CORE / NEO"].Sale_Price,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Adidas SPORT PERFORMANCE"].Sale_Price,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Nike"].Sale_Price,kde=False,rug=False)
plt.legend(labels=['Adidas ORIGINALS', 'Adidas CORE / NEO','Adidas SPORT PERFORMANCE', 'Nike'])
plt.show()

# 4) Rating by Brand

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(df[df["Brand"]=="Adidas ORIGINALS"].Rating,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Adidas CORE / NEO"].Rating,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Adidas SPORT PERFORMANCE"].Rating,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Nike"].Rating,kde=False,rug=False)
plt.legend(labels=['Adidas ORIGINALS', 'Adidas CORE / NEO','Adidas SPORT PERFORMANCE', 'Nike'])
plt.show()

# 5) Reviews by Brand

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(df[df["Brand"]=="Adidas ORIGINALS"].Reviews,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Adidas CORE / NEO"].Reviews,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Adidas SPORT PERFORMANCE"].Reviews,kde=False,rug=False)
sns.distplot(df[df["Brand"]=="Nike"].Reviews,kde=False,rug=False)
plt.legend(labels=['Adidas ORIGINALS', 'Adidas CORE / NEO','Adidas SPORT PERFORMANCE', 'Nike'])
plt.show()

# Correlation between "No_Listing_price", Rating and Reviews in NIKE.

In [ ]:
df_n0=df[(df['Listing_Price']==0)|(df['Brand']=='Nike')]
df_n0

In [ ]:
sns.pairplot(df_n0)

# A lot of NIKE Products have no reviews which have also no Listing_Price and no Rating. On the other hand, other products of NIKE get high rating compared to adidas products. It seems that there is big gap between fans and those who are not in NIKE.

# 2.Products Analysis

# 1) Top 30 Sales Price Products

In [ ]:
df_sp30=df.sort_values(by='Sale_Price',ascending=False).head(30)
sns.barplot(x = 'Sale_Price', y = 'Product Name', data = df_sp30, color = 'red')

# 2) Top 30 Listing Price Products

In [ ]:
df_lp30=df.sort_values(by='Listing_Price',ascending=False).head(30)
sns.barplot(x = 'Listing_Price', y = 'Product Name', data = df_lp30, color = 'green')

# 3. Correlation between Rating and other features

In [ ]:
df1=df[['Listing_Price','Sale_Price','Discount','Brand','Rating','Reviews']]

In [ ]:
df1

In [ ]:
df1 = pd.get_dummies(df1)

In [ ]:
df1

In [ ]:
sns.heatmap(df1.corr(),annot=True)

# Correlation between Listing_Price and Brand_Adidas_Original is 0.39.
# Correlation between Listing_Price and Brand_Adidas_Brand_Adidas SPORT PERFORMANCE is 0.23.
# Correlation between Sales_Price and Brand_Nike is 0.47.

# 4. Clustering

In [ ]:
df2=df[['Listing_Price','Sale_Price','Discount','Brand','Rating','Reviews']]
df2=df2.replace({'Adidas ORIGINALS':1, 'Adidas CORE / NEO':2,'Adidas SPORT PERFORMANCE':3,'Nike':4})

In [ ]:
df2

In [ ]:
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=4, random_state=10).fit(df2)

In [ ]:
labels = kmeans_model.labels_

In [ ]:
color_codes = {0:'red', 1:'blue', 2:'black',3:'yellow'}
colors = [color_codes[x] for x in labels]

In [ ]:
plotting.scatter_matrix(df2[df2.columns[1:]], figsize=(15,15), color=colors, alpha=0.8, diagonal='kde')
plt.show()

In [ ]:
df['cluster']=pd.DataFrame(labels)

In [ ]:
df_clus=df.groupby(['cluster','Brand'])['Product_ID'].count()
df_clus=df_clus.reset_index()

In [ ]:
df_clus

# 1) Brand, Sale_Price and cluster

In [ ]:
plt.figure(figsize=(12, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['Brand'],y=df['Sale_Price'],hue=df['cluster'],size=df['Sale_Price'],data=df,sizes=(1,200))
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

# 2) Brand, Rating and cluster

In [ ]:
plt.figure(figsize=(12, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['Brand'],y=df['Rating'],hue=df['cluster'],size=df['Rating'],data=df,sizes=(1,200))
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

# 3) Top 200 Sale_Price Product, Sale_Price and cluster

In [ ]:
df_sp200=df.sort_values(by='Sale_Price',ascending=False).head(200)

In [ ]:
plt.figure(figsize=(12, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df_sp200['Product Name'],y=df_sp200['Sale_Price'],hue=df_sp200['cluster'],size=df_sp200['Sale_Price'],data=df_sp200,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

# 4) Top 200 Sale_Price Product, Rating and cluster

In [ ]:
plt.figure(figsize=(12, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df_sp200['Product Name'],y=df_sp200['Rating'],hue=df_sp200['cluster'],size=df_sp200['Rating'],data=df_sp200,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

# 5) Lowest 200 Sale_Price Product, Sale_Price and cluster

In [ ]:
df_spt200=df.sort_values(by='Sale_Price',ascending=False).tail(200)

In [ ]:
plt.figure(figsize=(12, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df_spt200['Product Name'],y=df_spt200['Sale_Price'],hue=df_spt200['cluster'],size=df_spt200['Sale_Price'],data=df_spt200,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

# 6) Lowest 200 Sale_Price Product, Rating and cluster

In [ ]:
plt.figure(figsize=(12, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df_spt200['Product Name'],y=df_spt200['Rating'],hue=df_spt200['cluster'],size=df_spt200['Rating'],data=df_spt200,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

# Sale_Price plays very important role for cluter. On the other hand, Rating is distributed to each cluster, each Brand and each Product. It means that Price is common index for people to think about shoe Brand, adidas and NIKE. But Rating may depend on each preferenace, for example,comfotability.fittness,design etc....